In [1]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from numba import jit
import pstats
import heapq
import time
import cProfile
import pandas as pd
import numpy as np
import pandas as pd
from ast import literal_eval
from pathlib import Path
from pprint import pprint
from sympy.combinatorics import Permutation

puzzle_info_path = 'puzzle_info.csv'
puzzles_path = 'puzzles.csv'
sample_submission_path = 'sample_submission.csv'
my_submission_path = 'submission.csv'

puzzle_info_df = pd.read_csv(puzzle_info_path)
puzzles_df = pd.read_csv(puzzles_path)
sample_submission_df = pd.read_csv(sample_submission_path)
my_submission_df = pd.read_csv(my_submission_path)

In [2]:
puzzles_df[puzzles_df["puzzle_type"] == 'globe_1/8']
# Two types: permutation & colors

,id,puzzle_type,solution_state,initial_state,num_wildcards
338,338,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,I;P;O;A;A;D;F;L;J;M;G;M;P;F;E;J;E;B;O;G;H;D;N;...,0
339,339,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,K;I;C;B;M;J;D;G;E;H;N;E;F;I;D;K;L;P;A;O;B;N;J;...,0
340,340,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,M;O;L;M;P;F;L;B;I;G;E;J;A;G;O;I;C;H;D;P;B;A;N;...,0
341,341,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,P;O;G;J;A;L;F;C;D;B;P;N;F;M;I;G;O;M;H;H;E;N;K;...,0
342,342,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,F;H;O;M;C;J;K;I;J;L;B;N;B;G;K;H;D;P;A;D;E;N;P;...,2
343,343,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N21;N1;N9;N20;N30;N8;N25;N26;N11;N2;N16;N27;N4...,0
344,344,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N30;N0;N22;N15;N24;N19;N14;N18;N31;N6;N23;N16;...,0
345,345,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N19;N11;N22;N29;N15;N7;N14;N16;N0;N18;N9;N24;N...,0
346,346,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N9;N7;N17;N11;N30;N14;N5;N0;N16;N31;N6;N4;N1;N...,0
347,347,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N26;N10;N17;N24;N11;N3;N14;N16;N23;N2;N21;N1;N...,0


# Solving 2xM

In [3]:
idx = 343
N = 1+1
M = int(2*8)

In [4]:
def print_globe(b):
    print(np.asarray(b).reshape(2, M))

apply_move = {}

def set_moves(verbose = False): # Assumes M is reset
    test_globe = np.array([i for i in range(2*M)])
    r0_perm = np.array([i for i in range(2*M)])
    r1_perm = np.array([i for i in range(2*M)])
    f0_perm = np.array([i for i in range(2*M)])
    
    r0_perm[M-1] = 0
    for i in range(0, M-1):
        r0_perm[i] = i+1
    
    r1_perm[2 * M - 1] = M
    for i in range(M, 2*M-1):
        r1_perm[i] = i+1
    
    for i in range(M//2):
        f0_perm[i] = M + M//2 - 1 - i
    
    for i in range(M, M + M//2):
        f0_perm[i] = M//2-1 - (i-M)
    
    nr0_perm = np.array([i for i in range(2*M)])
    nr1_perm = np.array([i for i in range(2*M)])
    
    for i in range(M-1):
        nr0_perm = nr0_perm[r0_perm]
    
    for i in range(M-1):
        nr1_perm = nr1_perm[r1_perm]

    def r0(arr, moves):
        moves.append('r0')
        arr = arr[r0_perm]
        return arr
    
    def nr0(arr, moves):
        moves.append('-r0')
        arr = arr[nr0_perm]
        return arr
    
    def r1(arr, moves):
        moves.append('r1')
        arr = arr[r1_perm]
        return arr
    
    def nr1(arr, moves):
        moves.append('-r1')
        arr = arr[nr1_perm]
        return arr
    
    def f0(arr, moves):
        moves.append('f0')
        arr = arr[f0_perm]
        return arr
    
    apply_move['r0'] = r0
    apply_move['-r0'] = nr0
    apply_move['r1'] = r1
    apply_move['-r1'] = nr1
    apply_move['f0'] = f0

    def create_f_function(i, cur_perm):
        return lambda arr, moves: (moves.append('f' + str(i)), arr[cur_perm])[1]

    for i in range(1, M):
        cur_perm = np.array([i for i in range(2*M)])
    
        for j in range(i):
            cur_perm = cur_perm[r0_perm]
            cur_perm = cur_perm[r1_perm]
    
        cur_perm = cur_perm[f0_perm]
    
        for j in range(i):
            cur_perm = cur_perm[nr0_perm]
            cur_perm = cur_perm[nr1_perm]
            
        apply_move['f' + str(i)] = create_f_function(i, cur_perm)

    if verbose:
        print("r0")
        print_globe(r0(test_globe, []))
        print("nr0")
        print_globe(nr0(test_globe, []))
        print("r1")
        print_globe(r1(test_globe, []))
        print("f0")
        print_globe(f0(test_globe, []))
        print_globe(apply_move['f1'](test_globe, []))
        print_globe(apply_move['f2'](test_globe, []))

    set_half_turn_moves(verbose)

def set_half_turn_moves(verbose):
    test_globe = np.array([i for i in range(2*M)])
    def half_turn_ccw(arr, moves):
        for i in range(M//2):
            arr = apply_move['-r0'](arr, moves)
        
        arr = apply_move['f0'](arr, moves)
        arr = apply_move['r0'](arr, moves)
        arr = apply_move['f0'](arr, moves)
        
        for i in range(M//2):
            arr = apply_move['r0'](arr, moves)
        return arr
    
    def half_turn_cw(arr, moves):
        for i in range(M//2):
            arr = apply_move['-r0'](arr, moves)
        
        arr = apply_move['f0'](arr, moves)
        arr = apply_move['-r0'](arr, moves)
        arr = apply_move['f0'](arr, moves)
        
        for i in range(M//2):
            arr = apply_move['r0'](arr, moves)
        return arr
    
    apply_move['half_turn'] = half_turn_ccw
    apply_move['rev_half_turn'] = half_turn_cw

    def top_right_cw(arr, moves):
        arr = apply_move['f0'](arr, moves)
        arr = apply_move['-r0'](arr, moves)
        arr = apply_move['f0'](arr, moves)
        return arr
    
    def top_right_ccw(arr, moves):
        arr = apply_move['f0'](arr, moves)
        arr = apply_move['r0'](arr, moves)
        arr = apply_move['f0'](arr, moves)
        return arr
    
    def top_left_ccw(arr, moves):
        arr = apply_move['f0'](arr, moves)
        arr = apply_move['-r1'](arr, moves)
        arr = apply_move['f0'](arr, moves)
        return arr
    
    def top_left_cw(arr, moves):
        arr = apply_move['f0'](arr, moves)
        arr = apply_move['r1'](arr, moves)
        arr = apply_move['f0'](arr, moves)
        return arr
    
    apply_move['top_right_cw'] = top_right_cw
    apply_move['top_right_ccw'] = top_right_ccw
    apply_move['top_left_cw'] = top_left_cw
    apply_move['top_left_ccw'] = top_left_ccw

    if verbose:
        print_globe(half_turn_ccw(test_globe, []))
        print_globe(half_turn_cw(test_globe, []))
        print_globe(top_right_cw(test_globe, []))
        print_globe(top_right_ccw(test_globe, []))
        print_globe(top_left_cw(test_globe, []))
        print_globe(top_left_ccw(test_globe, []))

set_moves(True)

r0
[[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15  0]
 [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]
nr0
[[15  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]
r1
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 16]]
f0
[[23 22 21 20 19 18 17 16  8  9 10 11 12 13 14 15]
 [ 7  6  5  4  3  2  1  0 24 25 26 27 28 29 30 31]]
[[ 0 24 23 22 21 20 19 18 17  9 10 11 12 13 14 15]
 [16  8  7  6  5  4  3  2  1 25 26 27 28 29 30 31]]
[[ 0  1 25 24 23 22 21 20 19 18 10 11 12 13 14 15]
 [16 17  9  8  7  6  5  4  3  2 26 27 28 29 30 31]]
[[ 1  2  3  4  5  6  7 23  8  9 10 11 12 13 14 15]
 [ 0 16 17 18 19 20 21 22 24 25 26 27 28 29 30 31]]
[[16  0  1  2  3  4  5  6  8  9 10 11 12 13 14 15]
 [17 18 19 20 21 22 23  7 24 25 26 27 28 29 30 31]]
[[ 0  1  2  3  4  5  6  7 16  8  9 10 11 12 13 14]
 [17 18 19 20 21 22 23 15 24 25 26 27 28 29 30 31]]
[[ 0  1  2  3  4  5  6  7  9 10 11 12 13 14 15 23]
 [ 8 16 1

## Phase 1: Sort Elements Into Top/Bottom Arrays

In [5]:
test_globe = puzzles_df[puzzles_df["puzzle_type"] == "globe_1/8"]['initial_state'][idx]
test_globe = np.array([int(x[1:]) for x in test_globe.split(';') if x])
# print_globe(test_globe)
print(len(test_globe))

32


In [6]:
# test_globe = apply_move['f0'](test_globe, [])
# test_globe = apply_move['f9'](test_globe, [])
# test_globe = apply_move['r0'](test_globe, [])
# test_globe = apply_move['r0'](test_globe, [])
# test_globe = apply_move['f1'](test_globe, [])

In [7]:
print_globe(test_globe)

[[21  1  9 20 30  8 25 26 11  2 16 27  4 28  6 13]
 [17 31 12 18 19  0 29 10 22 14 24  7  3  5 15 23]]


In [8]:
moves = []

In [9]:
def is_moved_top_bottom(arr):
    for i in range(M):
        if arr[i] >= M:
            return False
    for i in range(M, 2*M):
        if arr[i] < M:
            return False
    return True

# change this to allow all M cyclic rotates
def move_top_bottom_helper(arr, moves):
    while not is_moved_top_bottom(arr):
        
        rotate_locs = [
            (M//2, M + M // 2 - 1, 'top_right_ccw'),
            (M-1, M, 'top_right_cw'),
            (M//2-1, M + M//2, 'top_left_cw'),
            (0, 2*M-1, 'top_left_ccw')
        ]

        all_costs = []

        for a, b, op_name in rotate_locs:
            mn_a, mn_b = int(1e9), int(1e9)
            top_ops, bot_ops = [], []
            
            for i in range(M):
                if arr[i] >= M: # TODO: Try no abs. Can only r0
                    if abs(i - a) < mn_a:
                        mn_a = abs(i-a)

                        if i < a: # rotate right
                            top_ops = ['-r0'] * (a - i)
                        else:
                            top_ops = ['r0'] * (i - a)

            for i in range(M, 2*M):
                if arr[i] < M:
                    if abs(i - b) < mn_b:
                        mn_b = abs(i - b)

                        if i < b:
                            bot_ops = ['-r1'] * (b - i)
                        else:
                            bot_ops = ['r1'] * (i - b)

            all_costs.append((mn_a + mn_b, top_ops + bot_ops, op_name))

        all_costs = list(sorted(all_costs))
        for i in all_costs[0][1]:
            arr = apply_move[i](arr, moves)
        arr = apply_move[all_costs[0][2]](arr, moves)
    return arr

print_globe(move_top_bottom_helper(test_globe, []))

[[13  1  9  8 11  2  4  5  6  3  7 10 14  0 12 15]
 [28 21 31 18 19 29 22 24 27 23 26 16 25 17 30 20]]


In [10]:
# def sort_top_bottom(arr, moves):
#     a = []
#     a_arr = sort_top_bottom_helper(arr, a)

#     b = ['f0', 'f' + str(M//2)] # untested
#     arr = apply_move['f0'](arr, [])
    
#     for i in range(M//2):
#         arr = apply_move['r0'](arr, [])
#         arr = apply_move['r1'](arr, [])

#     arr = apply_move['f0'](arr, [])

#     for i in range(M//2):
#         arr = apply_move['-r0'](arr, [])
#         arr = apply_move['-r1'](arr, [])

#     b_arr = sort_top_bottom_helper(arr, b)

#     if len(b) < len(a):
#         for i in b:
#             moves.append(i)
#         return b_arr

#     for i in a:
#         moves.append(i)
#     return a_arr

# test_globe = sort_top_bottom(test_globe, moves)
# test_globe = sort_top_bottom_helper(test_globe, moves)
# print_globe(test_globe)

In [11]:
test_globe = move_top_bottom_helper(test_globe, moves)
top_bot_moves = moves
top_bot_state = test_globe
# try only applying ex-red here

## Phase 2: Remove Inversions

In [12]:
test_globe = top_bot_state
moves = top_bot_moves

print_globe(test_globe)

[[13  1  9  8 11  2  4  5  6  3  7 10 14  0 12 15]
 [28 21 31 18 19 29 22 24 27 23 26 16 25 17 30 20]]


In [13]:
def merge_and_count(arr, temp, left, mid, right):
    i, j, k = left, mid + 1, left
    inv_count = 0

    while i <= mid and j <= right:
        if arr[i] <= arr[j]:
            temp[k] = arr[i]
            i += 1
        else:
            temp[k] = arr[j]
            inv_count += (mid - i + 1)
            j += 1
        k += 1

    while i <= mid:
        temp[k] = arr[i]
        k += 1
        i += 1

    while j <= right:
        temp[k] = arr[j]
        k += 1
        j += 1

    for i in range(left, right + 1):
        arr[i] = temp[i]

    return inv_count

def merge_sort_and_count(arr, temp, left, right):
    inv_count = 0
    if left < right:
        mid = (left + right) // 2
        inv_count += merge_sort_and_count(arr, temp, left, mid)
        inv_count += merge_sort_and_count(arr, temp, mid + 1, right)
        inv_count += merge_and_count(arr, temp, left, mid, right)
    return inv_count

def count_inversions(arr):
    n = len(arr)
    temp_arr = arr.copy()  # Create a copy of the array
    temp = [0] * n
    return merge_sort_and_count(temp_arr, temp, 0, n - 1)

print(count_inversions(test_globe[0:M]))
print(count_inversions(test_globe[M:2*M]))

49
66


In [14]:
temp_test_globe = np.array([i for i in range(2*M)])

# swaps i, i+1 for both top and bottom
def swap_same_idx(i, arr, moves):
    loc = M // 2 - 1
    if i < loc:
        i += M

    delta = i - loc

    arr = apply_move['f' + str(delta)](arr, moves)
    arr = apply_move['r0'](arr, moves)
    arr = apply_move['f' + str(delta)](arr, moves)

    arr = apply_move['r1'](arr, moves)
    arr = apply_move['r1'](arr, moves)

    plus_half_minus_one = (delta + loc) % M

    arr = apply_move['f' + str(plus_half_minus_one)](arr, moves)
    arr = apply_move['r0'](arr, moves)
    arr = apply_move['f' + str(plus_half_minus_one)](arr, moves)

    arr = apply_move['-r0'](arr, moves) # cuold remove, but should be handled by ex-red
    arr = apply_move['-r1'](arr, moves) # cuold remove, but should be handled by ex-red

    return arr

print_globe(temp_test_globe)
print_globe(swap_same_idx(1, temp_test_globe, []))

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]
[[ 0  2  1  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [16 18 17 19 20 21 22 23 24 25 26 27 28 29 30 31]]


In [15]:
def get_inversion_locs(arr):
    A, B = [], []
    for i in range(0, M-1):
        if (arr[i] > arr[i+1]):
            A.append(i)

    for i in range(M, 2*M-1):
        if (arr[i] > arr[i+1]):
            B.append(i - M)

    return (A, B)

# Could try: Non-circular min_abs_diff
def min_abs_diff(A, B):
    # A = A + [i + M for i in A]
    # B = B + [i + M for i in B]
    
    i, j = 0, 0
    min_diff = int(1e9)
    min_indices = (-1, -1)

    while i < len(A) and j < len(B):
        diff = abs(A[i] - B[j])

        if diff < min_diff:
            min_diff = diff
            min_indices = (A[i], B[j])

        if A[i] < B[j]:
            i += 1
        else:
            j += 1

    if min_indices[0] >= M:
        min_indices = (min_indices[0] - M, min_indices[1])

    if min_indices[1] >= M:
        min_indices = (min_indices[0], min_indices[1] - M)
        
    return min_indices

def is_array_zero_to_n(arr):
    n = len(arr) - 1
    return len(arr) == n + 1 and set(arr) == set(range(n + 1))

def check_swaps(arr, swaps):
    arr_copy = arr.copy()
    for swap in swaps:
        if 0 <= swap < len(arr_copy) - 1:
            arr_copy[swap], arr_copy[swap + 1] = arr_copy[swap + 1], arr_copy[swap]
            # print(arr_copy)
        else:
            return False
    return arr_copy == sorted(arr_copy)

# Phase 2, finding optimal order of swaps of one array to minimize the amount of rotations of other
# Minimize sum of absolute distances between swaps
def get_swaps_order(arr):
    swaps = []
    arr = [i % M for i in arr]

    mx = -1
    move_forward = []
    for i in range(len(arr)):
        if(arr[i] < mx):
             move_forward.append(i)
        mx = max(mx, arr[i])

    while move_forward:
        for i in range(len(move_forward)-1, -1, -1):
            if i != 0 and move_forward[i-1] == move_forward[i] - 1:
                continue

            swaps.append(move_forward[i] - 1)
            arr[move_forward[i]-1], arr[move_forward[i]] = arr[move_forward[i]], arr[move_forward[i]-1]
            move_forward[i] -= 1
            
            temp = -1
            for k in range(move_forward[i]):
                temp = max(temp, arr[k])

            
            if temp <= arr[move_forward[i]]:
                move_forward.pop(i)
                
            break

    if (not is_array_zero_to_n(arr)):
        print("get_swaps_order function is incorrect")
        # swaps = swaps + get_swaps_order(arr)
    
    return swaps

x = get_swaps_order([2,6,0,5,4,7,1,3])

In [16]:
def is_array_zero_to_n(arr):
    n = len(arr) - 1
    return len(arr) == n + 1 and set(arr) == set(range(n + 1))

def remove_inversions(arr, moves):
    # print("A")
    # print_globe(arr)

    # Phase 1: Both Have Inversions
    up_inversions, down_inversions = get_inversion_locs(arr)

    while up_inversions and down_inversions:
        up_i, down_i = min_abs_diff(up_inversions, down_inversions)

        if down_i < up_i: 
            diff = up_i - down_i # Could do: Try swapping K/2 left up, K/2 down right
            for k in range(diff):
                arr = apply_move['r0'](arr, moves)

            arr = swap_same_idx(down_i, arr, moves)

            for k in range(diff):
                arr = apply_move['-r0'](arr, moves)

        else:
            diff = down_i - up_i # Could do: Try swapping K/2 left up, K/2 down right
            for k in range(diff):
                arr = apply_move['r1'](arr, moves)

            arr = swap_same_idx(up_i, arr, moves)

            for k in range(diff):
                arr = apply_move['-r1'](arr, moves)

        up_inversions, down_inversions = get_inversion_locs(arr)
        

    # print("B")
    # print_globe(arr)
    
    # Phase 2: Only one has inversion (FIX)
    if (count_inversions(arr[M:2*M]) + count_inversions(arr[0:M])) % 2 != 0:
        print("Failure: Leftover Inversions should be even")
        return [-1]

    if up_inversions:
        up_swaps = get_swaps_order(arr[0:M])
        for k in range(0, len(up_swaps), 2):
            diff = (up_swaps[k] - up_swaps[k+1] + M) % M
            
            arr = swap_same_idx(up_swaps[k], arr, moves)
            
            for l in range(diff):
                arr = apply_move['r1'](arr, moves)
            arr = swap_same_idx(up_swaps[k+1], arr, moves)
            
        
    if down_inversions:
        down_swaps = get_swaps_order(arr[M:2*M])
        for k in range(0, len(down_swaps), 2):
            diff = (down_swaps[k] - down_swaps[k+1] + M) % M
            
            arr = swap_same_idx(down_swaps[k], arr, moves)
            
            for l in range(diff):
                arr = apply_move['r0'](arr, moves)

            arr = swap_same_idx(down_swaps[k+1], arr, moves)


    # Phase 3: Do some final rotations to get everything aligned right. Check correctness
    while arr[0] != 0:
        arr = apply_move['r0'](arr, moves)

    while arr[M] != M:
        arr = apply_move['r1'](arr, moves)
    
    # print("C")
    # print_globe(arr)

    return arr


# test_globe = apply_move['r0'](test_globe, [])

print_globe(test_globe)
remove_inversions(test_globe, [])

[[13  1  9  8 11  2  4  5  6  3  7 10 14  0 12 15]
 [28 21 31 18 19 29 22 24 27 23 26 16 25 17 30 20]]
Failure: Leftover Inversions should be even


[-1]

In [17]:
def sort_top_bot(arr, moves = []):
    res = [-1]

    for i in range(M-1):
        for j in range(M-1):
            cur_moves = []
            cur_arr = arr
            
            for k in range(i):
                cur_arr = apply_move['r0'](cur_arr, cur_moves)

            for k in range(j):
                cur_arr = apply_move['r1'](cur_arr, cur_moves)
            
            if (count_inversions(cur_arr[0:M]) % 2 != count_inversions(cur_arr[M:2*M]) % 2):
                continue

            cur_arr = remove_inversions(cur_arr, cur_moves)

            # TODO: Apply Exp-Red

            if -1 in res or len(cur_moves) < len(res):
                res = cur_moves
    
    if -1 in res:
        print("Failure: No Rotations where top and bottom have same parity inversions")

    return res

res = sort_top_bot(test_globe)

In [18]:
print(len(res))

623


In [19]:
test_globe_copy = test_globe
for i in res:
    test_globe_copy = apply_move[i](test_globe_copy, [])
print_globe(test_globe_copy)

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]


In [20]:
print(len(res))

623


In [21]:
moves = moves + res
print(len(moves))

667


In [22]:
test_globe = puzzles_df[puzzles_df["puzzle_type"] == "globe_1/8"]['initial_state'][idx]
test_globe = np.array([int(x[1:]) for x in test_globe.split(';') if x])
print_globe(test_globe)

[[21  1  9 20 30  8 25 26 11  2 16 27  4 28  6 13]
 [17 31 12 18 19  0 29 10 22 14 24  7  3  5 15 23]]


In [23]:
for i in moves:
    test_globe = apply_move[i](test_globe, [])
print_globe(test_globe)

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]


# Transition

In [58]:
def get_run_length_encoding(path):    
    rows = [[] for _ in range(2)]
    
    def expand_f(x):
        for i in range(2):
            rows[i].append(x)
            rows[i].append('f0')
            rows[i].append(M - x)
    
    for i in path:
        if 'f' in i:
            if '-' in i:
                expand_f(int(i[2:]))
            else:
                expand_f(int(i[1:]))
        else:
            for j in range(2):
                temp = 'r' + str(j)

                if temp == i:
                    rows[j].append(1)
                    
                temp = '-r' + str(j)

                if temp == i:
                    rows[j].append(M-1)

    Rrows = [[] for _ in range(2)]

    for r in range(2):
        run = 0
        for i in rows[r]:
            if type(i) == type(0):
                run += i
            else:
                Rrows[r].append(run)
                Rrows[r].append('f0')
                run = 0
            
        Rrows[r].append(run)
    
    return (Rrows[0], Rrows[1])

In [59]:
def check_moves(arr, moves):
    for i in moves:
        arr = apply_move[i](arr, [])
    return is_array_zero_to_n(arr)

def solve_2xM(arr):
    if not is_array_zero_to_n(list(sorted(arr))):
        print("Array not in correct format for 2xM Solve")
    M = len(arr) // 2

    # Move top/bot
    move_moves = []
    arr = move_top_bottom_helper(arr, move_moves)

    # Sort top/bot
    sort_moves = sort_top_bot(arr)

    # Combine moves
    res_moves = move_moves + sort_moves

    if not check_moves(arr, res_moves):
        print("solve_2xM: res_moves is incorrect")

    print("Sorting moves:", len(sort_moves), "Move top/bot moves:", len(move_moves))
    print("Before Ex-Red", len(res_moves))
    
    # res_moves = ex_red(res_moves)

    # print("After Ex-Red", len(res_moves))
    return res_moves
    

def solve_1xM(arr):
    print("TODO")


# test_globe = puzzles_df[puzzles_df["puzzle_type"] == "globe_1/8"]['initial_state'][343]
# test_globe = np.array([int(x[1:]) for x in test_globe.split(';') if x])
# print_globe(test_globe)
# temp = solve_2xM(test_globe)

# Solving N x M

In [125]:
# GLobe 1/16
# idx = 348
# N = 1+1
# M = int(2*16)

In [126]:
#Globe 3/33
idx = 395
N = 3+1 
M = int(2*33)

In [127]:
puzzles_df.iloc[idx]

id                                                                     395
puzzle_type                                                     globe_3/33
solution_state           N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...
initial_state            N12;N219;N227;N198;N4;N208;N214;N245;N56;N55;N...
num_wildcards                                                            0
parsed_initial_state     [10, 5837, 5577, 212, 14, 197, 204, 6307, 89, ...
parsed_solution_state    [7, 8, 9, 16, 14, 13, 25, 20, 27, 23, 29, 24, ...
Name: 395, dtype: object

In [139]:
# Make all encodings same length
def equalize_lengths(encodings):
    mx_length = 0
    for i in encodings:
        mx_length = max(mx_length, len(i))

    for i in range(len(encodings)):
        if(((mx_length - len(encodings[i]))/2) % 2 == 1):
            print("Failure: Odd number of F0s needed to allign rows!")

        while(len(encodings[i]) < mx_length):
            encodings[i].append('f0')
            encodings[i].append(0)

    return encodings

def is_int(x):
    return isinstance(x, int)

def combine_palindromic_rows(encodings):
    encodings = equalize_lengths(encodings)
    n, m = len(encodings), len(encodings[0]) # number of rows / length of run-length encoding of each row

    # Take modulo + Convert F0s to FXs
    for j in range(m):
        mn = int(1e9)
        for i in range(n):
            if is_int(encodings[i][j]): # Is rotation
                encodings[i][j] %= M
            else:
                mn = min(mn, encodings[i][j-1])
        
        # for i in range(n):  
        #     if not is_int(encodings[i][j]): # Is f0
        #         encodings[i][j-1] -= mn
        #         encodings[i][j] = 'f' + str(mn)
        #         encodings[i][j+1] += mn

    # Convert to inverse rotation when better
    moves = []
    for j in range(m):
        if not is_int(encodings[0][j]):
            moves.append(encodings[0][j])
            continue

        for i in range(n):
            if M - encodings[i][j] < encodings[i][j]:
                moves = moves + ['-r' + str(i)] * (M - encodings[i][j])
            else:
                moves = moves + ['r' + str(i)] * (encodings[i][j])

    # print(moves)
    # Remove duplicate FXs
    final_moves = []
    i = 0
    while i < len(moves):
        if i != len(moves) - 1 and ('f' in moves[i]) and moves[i] == moves[i+1]:
            i += 2
        else:
            final_moves.append(moves[i])
            i += 1
    
    return final_moves

In [140]:
def get_perm(start_state, end_state):
    if set(start_state) != set(end_state):
        print("Get Perm: set of values should be the same")

    sz = len(start_state)

    # final state has 0...2*M-1

    arr = end_state.copy()
    res = []

    for i in range(sz):
        for j in range(sz):
            if(start_state[i] == end_state[j]):
                res.append(j)

                # TODO: Smartly handle duplicates
                end_state[j] = -1
                break

    res = np.array(res)
    return res

# test_initial_state = puzzles_df['initial_state'][idx].split(';') <== only for 2/M
# test_final_state = puzzles_df['solution_state'][idx].split(';')
# print_globe(test_initial_state)
# print_globe(test_final_state)
# print_globe(get_perm(test_initial_state,test_final_state ))

In [141]:
# redefine moves

initial_state = puzzles_df['initial_state'][idx].split(';')
solution_state = puzzles_df['solution_state'][idx].split(';')

all_encodings = []
top_half_moves = []
bot_half_moves = []

for i in range(N):
    if N-1-i < i:
        break
        
    cur_start_state = initial_state[i*M : i*M + M] + initial_state[(N-1-i) * M : (N-1-i) * M + M]
    cur_end_state = solution_state[i*M : i*M + M] + solution_state[(N-1-i) * M : (N-1-i) * M + M]

    cur_arr = get_perm(cur_start_state, cur_end_state)

    print_globe(cur_arr)

    if N-1-i == i:
        # Set moves for 1xM separately
        cur_moves = solve_1xM(cur_arr)
        # TODO: Append moves to top half
        continue

    set_moves()
    cur_moves = solve_2xM(cur_arr) 
    
    cur_top_moves, cur_bot_moves = get_run_length_encoding(cur_moves)

    top_half_moves.append(cur_top_moves)
    bot_half_moves.append(cur_bot_moves)
    
all_encodings = top_half_moves + bot_half_moves[::-1]

[[ 12  87  95  66   4  76  82 113  56  55  88  32  44 100  53 128  36 119
   50 102  39  68   1 126 104   7 124  74 106  62  65  13  78 112 103  33
   70  30  63  11  17 117   8  54  92  45  99  10 111   3 107  19   9  83
   86  52  96  42  38  79  48  72  16  77  91  24]
 [ 98  69 120 125  31 118  43  97  14  80 122  41  85  15  59  18 105  20
   73  51  84  27  37  21 101  22  61  35  90  28   0   6 129  34  49  89
   60   5 114  75 115  40 109  46 116  94  64 130  57  81 123 127  67  25
   47  23  58 121 131 110   2  71  26  93  29 108]]
Sorting moves: 11673 Move top/bot moves: 223
Before Ex-Red 11896
[[129  99  85 122  17  36 123  95   4  49  32  18 110  25  91  68  61  37
   23  82  88 112 111  11   8  63 118 108  94  80  42 102  16  24  27  81
  131  31  19  75 109 125 101  77   7  87  79 124  67 115   1  41 119  39
   15  53  74  90  92 127  84  66   0  26  65 114]
 [ 52  20 130 105  13  86   5  72  59  71 120  22  76  70  93  97   2  33
    9  43  57  62  14  56 113  83  35 117

In [142]:
# print(all_encodings)

In [143]:
res_moves = combine_palindromic_rows(all_encodings)

In [144]:
print(len(res_moves))

248960


In [145]:
# res_moves

# Checking Moves With Official

In [146]:
def getInversePerm(arr):
    # gets the inverse move for a certain move
    res = [0 for i in range(len(arr))]
    for i in range(len(arr)):
        res[arr[i]] = i
    return res
    
# type : (np.array(move_perm_i), np.array(name_i))
puz_info = {}

# type : {move : perm}
move_to_perm = {}

for i in range(len(puzzle_info_df)):
    puz_info[puzzle_info_df['puzzle_type'][i]] = [[], []]
    move_to_perm[puzzle_info_df['puzzle_type'][i]] = {}
    
    for j in puzzle_info_df['allowed_moves'][i].keys():
        puz_info[puzzle_info_df['puzzle_type'][i]][1].append(j)
        puz_info[puzzle_info_df['puzzle_type'][i]][0].append(np.array(puzzle_info_df['allowed_moves'][i][j]))

        puz_info[puzzle_info_df['puzzle_type'][i]][1].append(str('-' + j)) # might be the opposite
        puz_info[puzzle_info_df['puzzle_type'][i]][0].append(np.array(getInversePerm(puzzle_info_df['allowed_moves'][i][j])))

        move_to_perm[puzzle_info_df['puzzle_type'][i]][str('-' + j)] = np.array(getInversePerm(puzzle_info_df['allowed_moves'][i][j]))
        move_to_perm[puzzle_info_df['puzzle_type'][i]][j] = np.array(puzzle_info_df['allowed_moves'][i][j])

# move_to_perm['cube_2/2/2']

In [147]:
def check_official(moves, idx):
    puzzle_type = puzzles_df['puzzle_type'][idx]
    initial_state = np.array(puzzles_df['initial_state'][idx].split(';'))
    solution_state = np.array(puzzles_df['solution_state'][idx].split(';'))

    official_moves = move_to_perm[puzzle_type]
    
    for i in moves:
        initial_state = initial_state[official_moves[i]]

    return all(solution_state == initial_state)

check_official(res_moves, idx)

True

# Submissions #

The submission format requires moves in a sequence to be delimited by a period `.`. Indicate the inverse of a named move with a preceeding `-`. Moves are applied from left to right.

The `sample_submission.csv` contains a baseline solution for each puzzle.

Let's check that the given sequence for the first puzzle is actually a solution.